# การสกัดฟีเจอร์ (Feature Extraction) สำหรับการตรวจจับการไอ

Notebook นี้ทำหน้าที่เป็นรากฐานว่าฟีเจอร์ใดบ้างที่ถูกสกัดจาก multimodal biosignals (สัญญาณชีวภาพแบบหลายรูปแบบ) (เสียง + เซ็นเซอร์ IMU) สำหรับการตรวจจับการไอโดยใช้ Machine Learning และเหตุผลที่ต้องสกัดฟีเจอร์เหล่านั้น

## การกำหนดค่า (Configuration)

- **Window Size (ขนาดหน้าต่าง)**: 0.4 วินาที (ตัวอย่างเสียง 6400 ตัวอย่างที่ 16kHz, ตัวอย่าง IMU 40 ตัวอย่างที่ 100Hz)
- **Data augmentation (การเพิ่มข้อมูล)**: การเลื่อนเวลาแบบสุ่ม (random temporal shifts) กำหนดค่า aug_factor=2 เพื่อเพิ่มความหลากหลายของชุดข้อมูล

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats, signal
import librosa
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

if os.path.exists("/kaggle/usr/lib/"):
    # Load from Kaggle as utility scripts
    from edge_ai_cough_count_helpers import * # pyright: ignore[reportMissingImports]
    from edge_ai_cough_count_dataset_gen import * # pyright: ignore[reportMissingImports]
    from edge_ai_cough_count_features import extract_audio_features, extract_imu_features # pyright: ignore[reportMissingImports]
else:
    # Add src directory to path
    sys.path.append(os.path.abspath('../src'))
    from helpers import *
    from dataset_gen import *
    from features import extract_audio_features, extract_imu_features

print("✓ All imports successful")

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Constants (ค่าคงที่) from paper
WINDOW_LEN = 0.4  # 0.4 second windows
AUG_FACTOR = 2    # Data augmentation factor

## เหตุผลที่ต้องมีการสกัดฟีเจอร์ (Why Feature Extraction?)

คุณอาจสงสัยว่า: **ทำไมไม่ป้อนสัญญาณเสียงและสัญญาณ IMU ดิบเข้าไปในโมเดล Machine Learning โดยตรงเลยล่ะ?**

เหตุผลที่การสกัดฟีเจอร์มีความสำคัญคือ:

1. **การลดขนาด (Size reduction)**: หน้าต่างขนาด 0.4 วินาที มีตัวอย่างเสียง 6,400 ตัวอย่าง แทนที่จะประมวลผลค่าทั้งหมด 6,400 ค่า เราจะสกัดเพียง 65 ฟีเจอร์ที่มีความหมาย ซึ่งจับ "แก่นแท้" ของเสียงได้

2. **การลดสัญญาณรบกวน (Noise reduction)**: สัญญาณดิบมีข้อมูลที่ไม่เกี่ยวข้องและสัญญาณรบกวนจำนวนมาก ฟีเจอร์จะมุ่งเน้นไปที่สิ่งที่สำคัญ นั่นคือรูปแบบที่แยกความแตกต่างระหว่างการไอกับเสียงอื่น ๆ

3. **การเรียนรู้ที่ดีขึ้น (Better learning)**: โมเดล Machine Learning จะเรียนรู้ได้เร็วและแม่นยำกว่าจากฟีเจอร์ที่กระชับและมีความหมาย เมื่อเทียบกับการใช้ค่าสัญญาณดิบหลายพันค่า

**การเปรียบเทียบ**: ลองจินตนาการถึงการอธิบายบุคคล แทนที่จะแสดงรูปภาพความละเอียดสูงเต็มรูปแบบ (สัญญาณดิบ) คุณจะอธิบายฟีเจอร์สำคัญ ๆ เช่น ส่วนสูง สีตา อายุ รูปร่าง (ฟีเจอร์ที่ถูกสกัด) ทั้งสองอย่างสามารถสื่อสารข้อมูลได้ แต่ฟีเจอร์จะกระชับกว่าและเน้นไปที่สิ่งที่สำคัญ

ใน Notebook นี้ เราจะสกัด **105 ฟีเจอร์** ที่จับลักษณะเฉพาะของเสียงและการเคลื่อนไหวของการไอ

## ภาพรวมของหมวดหมู่ฟีเจอร์

ฟีเจอร์ 105 ตัวของเราถูกจัดระเบียบเป็นสองกลุ่มหลัก:

| หมวดหมู่ | จำนวน | สิ่งที่จับได้ | ตัวอย่างการใช้งาน |
|----------|-------|------------------|------------------|
| **Audio Features** | **65** | ลักษณะของเสียงจากไมโครโฟน | |
| → MFCC | 52 | "พื้นผิว (texture)" หรือ "คุณภาพเสียง (timbre)" ของเสียง | แยกแยะการไอกับการพูด |
| → Spectral | 10 | การกระจายความถี่ (frequency distribution) | เสียงสูงเทียบกับเสียงต่ำ |
| → Time-domain | 3 | ความดัง (loudness) และความคมชัด (sharpness) | เสียงที่เกิดขึ้นอย่างกะทันหันเทียบกับเสียงที่ค่อยเป็นค่อยไป |
| **IMU Features** | **40** | การเคลื่อนไหวของร่างกายจากเซ็นเซอร์ | |
| → Accelerometer | 20 | การเคลื่อนที่เชิงเส้น (การเคลื่อนไหวของหน้าอก/ร่างกาย) | ตรวจจับการเคลื่อนไหวทางกายภาพของการไอ |
| → Gyroscope | 20 | การเคลื่อนที่แบบหมุน (ศีรษะ/คอ) | ตรวจจับการเคลื่อนไหวของศีรษะขณะไอ |
| **รวม (Total)** | **105** | ลายเซ็นหลายรูปแบบที่สมบูรณ์ (Complete multimodal signature) | การตรวจจับการไอที่แม่นยำและทนทาน |

### เหตุผลที่ต้องใช้ทั้งเซ็นเซอร์เสียง (Audio) และเซ็นเซอร์ตรวจจับการเคลื่อนไหว (Motion Sensors)?

**การรับรู้แบบหลายรูปแบบ (Multimodal sensing) มีความแม่นยำและทนทานมากกว่าการใช้รูปแบบใดรูปแบบหนึ่งเพียงอย่างเดียว:**

- **เมื่อเสียงมีปัญหา**: สภาพแวดล้อมที่มีเสียงรบกวน (เพลง, การจราจร, คนอื่นพูดคุย)
  - IMU ยังคงจับภาพการเคลื่อนไหวทางกายภาพของการไอได้
  
- **เมื่อ IMU มีปัญหา**: การนั่งนิ่ง, การไอเบา ๆ, การเคลื่อนไหวของร่างกายอื่น ๆ
  - เสียงจะจับลักษณะเฉพาะของเสียงไอ
  
- **เมื่อทำงานร่วมกัน**: การรวมกันมีความแม่นยำมากกว่าการใช้เซ็นเซอร์เดียวอย่างมาก

ในคำแนะนำต่อไปนี้ คุณจะได้เห็นว่าฟีเจอร์แต่ละหมวดหมู่ทำงานอย่างไร และเหตุใดฟีเจอร์เหล่านี้จึงช่วยในการตรวจจับการไอ!

## ทำความเข้าใจเกี่ยวกับ Sampling (การสุ่มตัวอย่าง) และ Sample Rate (อัตราการสุ่มตัวอย่าง)

ก่อนที่เราจะสามารถสกัดฟีเจอร์จากสัญญาณเสียงและ IMU ได้ เราจำเป็นต้องเข้าใจว่า ในโลกแห่งความเป็นจริง **สัญญาณต่อเนื่อง (continous)** เหล่านี้ กลายเป็น **ข้อมูล digital แบบไม่ต่อเนื่อง (discrete digital data)** ที่คอมพิวเตอร์สามารถประมวลผลได้อย่างไร

### Sampling (การสุ่มตัวอย่าง) คืออะไร?

**Sampling (การสุ่มตัวอย่าง)** คือกระบวนการแปลงสัญญาณ analog แบบต่อเนื่อง (เช่น คลื่นเสียงหรือการเคลื่อนไหว) ให้เป็นสัญญาณ digital แบบไม่ต่อเนื่อง โดยการวัดค่าในช่วงเวลาที่สม่ำเสมอ

**การเปรียบเทียบ**: ลองจินตนาการว่าคุณกำลังดูรถยนต์ขับผ่านไป:
- **ต่อเนื่อง (analog)**: รถเคลื่อนที่อย่างราบรื่นตลอดพื้นที่
- **ไม่ต่อเนื่อง (digital)**: คุณถ่ายภาพนิ่งทุก ๆ 0.1 วินาทีเพื่อติดตามตำแหน่งของรถ

ภาพนิ่งเหล่านี้คือ **samples (ตัวอย่าง)** ของคุณ และความถี่ที่คุณถ่ายภาพนิ่งคือ **sampling rate (อัตราการสุ่มตัวอย่าง)** ของคุณ

### Sample Rate หรือ Sampling Frequency (อัตราการสุ่มตัวอย่าง) คืออะไร?

**Sample rate (อัตราการสุ่มตัวอย่าง)** เรียกอีกอย่างว่า **sampling frequency (ความถี่ในการสุ่มตัวอย่าง)** คือจำนวน samples ที่ถูกนำมาต่อวินาที วัดเป็นหน่วย **เฮิรตซ์ (Hertz, Hz)**

**ตัวอย่าง:**
- **16,000 Hz** หมายถึง 16,000 samples ต่อวินาที
- **100 Hz** หมายถึง 100 samples ต่อวินาที

**Sample Rate ที่สูงขึ้น** = จำนวนภาพนิ่งต่อวินาทีมากขึ้น = การแสดงสัญญาณที่มีรายละเอียดมากขึ้น

### เซ็นเซอร์ของเราและ Sample Rate

ในตรวจจับการไอครั้งนี้ เราใช้เซ็นเซอร์สองประเภทที่มี Sample Rate ที่แตกต่างกัน:

| Sensor | Sample Rate | Samples per Second | สิ่งที่จับได้ |
|--------|-------------|-------------------|------------------|
| **Microphone** | 16,000 Hz | 16,000 | คลื่นเสียง (ไอ, การพูด, เสียงรอบข้าง) |
| **IMU (Accelerometer + Gyroscope)** | 100 Hz | 100 | การเคลื่อนไหวของร่างกาย (การเคลื่อนไหวของหน้าอก, การหมุนของศีรษะ) |

**ทำไมอัตราถึงแตกต่างกัน?**
- **เสียง (Audio)** เปลี่ยนแปลงอย่างรวดเร็วมาก (ความถี่เสียงอยู่ในช่วง 20 Hz ถึง 20,000 Hz) ดังนั้นเราจึงต้องการอัตราการสุ่มตัวอย่างที่สูงเพื่อจับภาพได้อย่างแม่นยำ
- **การเคลื่อนไหวของร่างกาย (Body motion)** เปลี่ยนแปลงช้ากว่าคลื่นเสียงมาก ดังนั้น 100 ตัวอย่าง/วินาทีจึงเพียงพอ

### จากโลกแห่งความเป็นจริงสู่ Digital Samples (From Real World to Digital Samples)

สิ่งที่เกิดขึ้นจริงคือ:

**เมื่อคุณไอ:**

1. **Microphone** วัดความดันอากาศ 16,000 ครั้งต่อวินาที → สร้างอาร์เรย์ของตัวเลข 16,000 ตัวต่อวินาที
2. **Accelerometer** วัดความเร่งของหน้าอก 100 ครั้งต่อวินาที → สร้างอาร์เรย์ของตัวเลข 100 ตัวต่อวินาที
3. **Gyroscope** วัดการหมุนของร่างกาย 100 ครั้งต่อวินาที → สร้างอาร์เรย์ของตัวเลข 100 ตัวต่อวินาที

**การบันทึก 1 วินาทีจะสร้าง:**
- เสียง (Audio): 16,000 ค่า (ตัวเลขที่แสดงแอมพลิจูดของเสียง)
- IMU: 100 ค่าสำหรับแต่ละช่องสัญญาณจากทั้งหมด 6 ช่องสัญญาณ (Accelerometer 3 แกน + Gyroscope 3 แกน) = รวม 600 ค่า

เมื่อนำตัวเลขเหล่านี้รวมกันเป็น Array ต่อ ๆ กัน เราจะใช้ในการสกัดฟีเจอร์!

### สิ่งนี้สำคัญอย่างไร?

การทำความเข้าใจ Sample Rate มีความสำคัญเนื่องจาก:
1. มันบอกเราว่าเรากำลังทำงานกับ **ข้อมูลจำนวนเท่าใด**
2. มันกำหนด **การแปลงเวลาเป็นตัวอย่าง** (time-to-sample conversion) (เช่น "0.5 วินาที = ตัวอย่างเสียง 8,000 ตัวอย่าง")
3. มันส่งผลต่อ **ขนาดหน้าต่าง (window sizes)** สำหรับ Machine Learning (อธิบายในส่วนถัดไป)

เรามาตรวจสอบ Sample Rate ของข้อมูลทั้ง 2 ชุดกัน:

In [ ]:
print(f"Audio sample rate: {FS_AUDIO} Hz")
print(f"IMU sample rate: {FS_IMU} Hz")

## ทำความเข้าใจ Windowing (การแบ่งเป็นหน้าต่าง) - Fixed-Length Segments (ส่วนที่มีความยาวคงที่)

### ทำไมเราถึงต้องมี Window (หน้าต่าง)?

โมเดล Machine Learning ต้องการ **input ที่มีขนาดคงที่ (fixed-size inputs)** แต่บันทึก (recordings) ของเรามีความยาวแตกต่างกัน:
- บางบันทึกมีความยาว 10 วินาที
- บันทึกอื่น ๆ มีความยาว 30 วินาทีหรือนานกว่านั้น
- การไอเพียงครั้งเดียวอาจใช้เวลา 0.3 วินาที แต่อีกครั้งอาจใช้เวลา 0.6 วินาที

**วิธีแก้ปัญหา**: สกัด **Windows (หน้าต่าง)** หรือส่วนที่มีความยาวคงที่จากไฟล์ที่บันทึกไว้

### Window คืออะไร?

**Window (หน้าต่าง)** คือส่วนของสัญญาณที่มีความยาวคงที่ เหมือนกับการตัดภาพยนตร์ออกเป็นคลิปที่มีความยาว 10 วินาทีเท่ากันทุกคลิป

จากเอกสารต้นฉบับ:
- **ความยาวของ Window (Window length)**: 0.4 วินาที
- **ทำไมต้อง 0.4 วินาที?** นี่เป็นระยะเวลาที่ยาวพอที่จะจับภาพการไอที่สมบูรณ์ได้ (โดยทั่วไปคือ 0.2-0.5 วินาที) แต่สั้นพอที่จะมีประสิทธิภาพสำหรับ ML

### ความสัมพันธ์ระหว่าง Window Length ↔ Sample Count

จำนวน Samples ใน Window ขึ้นอยู่กับ **Sampling Frequency**:

**สูตร**: `number_of_samples = window_length_seconds × sampling_frequency`

**สำหรับเสียง (16,000 Hz):**
- 0.4 วินาที × 16,000 Samples/วินาที = **6,400 Samples**

**สำหรับ IMU (100 Hz):**
- 0.4 วินาที × 100 Samples/วินาที = **40 Samples**

**การเปรียบเทียบ**: หากคุณบันทึกวิดีโอที่ 30 เฟรมต่อวินาที คลิป 2 วินาทีจะมี 2 × 30 = 60 เฟรม เป็นแนวคิดเดียวกัน!

In [ ]:
def extract_sample_window(
        sample: np.ndarray,
        frequency: int, 
        window_duration: float,
        start_point_index = None,
        start_point_time = None,
    ):
    """
    Extract features from the given NumPy array, the window duration, and optionally, the starting point.
    The starting point can either be an index of the sample, or a the time that sample should start,
    which in this case will calculate the sample index automatically.

    ## Example:
    - Given the sample rate: 16000 Hz
    - If window starts at 0.5s → 0.5 * 16000 = 8000th sample
    - The window length to extract is 0.4s → 0.4 × 16000 = 6400 samples
    - Thus, the sample region to extract is feature[8000: 8000 + 6400]
    """
    window_length = int(window_duration * frequency)
    if start_point_index is None:
        if start_point_time is not None:
            # Calculate the sample start point from the given time
            start_point_index = int(start_point_time * frequency)
        else:
            raise ValueError("You must provide either start_point_index or start_point_time.")
    else:
        # Clamp to [0, sample_length - window_length]
        start_point_index = min(max(0, start_point_index), len(sample) - window_length)
    
    # Slice the sample using NumPy slicing
    return sample[start_point_index: start_point_index + window_length]

### Window สองประเภทที่เราจะสกัด

ในเอกสารนี้ เราจำเป็นต้องสกัดเสียงไอและเสียงที่ไม่ใช่ไอ ดังนั้น เราจะมี Window สองประเภท

1. **Cough Windows (หน้าต่างการไอ)**
   - เรารู้เวลาเริ่มต้น/สิ้นสุดที่แน่นอนจาก `ground_truth.json` ดังนั้นเราจึงสามารถใช้เป็นข้อมูลอ้างอิงได้
   - เราจะสกัด Window เหล่านี้โดยมีจุดศูนย์กลางอยู่ที่เหตุการณ์การไอที่มีการกำกับ (annotation) ไว้
   - Window จะรวมถึงการไอ ซึ่งอาจถูกสุ่มเลื่อนเพื่อใช้ในการเพิ่มข้อมูล (Data augmentation)

2. **Non-cough windows (หน้าต่างที่ไม่ใช่การไอ)**:
   - ไม่มี Ground Truth (ข้อมูลความจริง) เพราะเราไม่ได้ไอ!
   - เราจะสุ่มตัวอย่างส่วน (segments) จากเสียงที่ไม่ใช่การไอ (เช่น เสียงหัวเราะ, เสียงกระแอม, เสียงหายใจ)
   - ใช้เป็น Negative Examples (ตัวอย่างเชิงลบ) (Label = 0)

In [ ]:
def extract_subject_window(audio, imu, start_point_time=None, start_point_index=None):
    """Extract audio and IMU window from the given window start point"""
    audio_window = extract_sample_window(audio, FS_AUDIO, WINDOW_LEN, start_point_index, start_point_time)

    # For IMU features, we need to stack all 6 IMU channels into a 2D array
    # Channels: [Accel_x, Accel_y, Accel_z, Gyro_Y, Gyro_P, Gyro_R]
    imu_window = np.column_stack([
        extract_sample_window(feature, FS_IMU, WINDOW_LEN, start_point_index, start_point_time)
        for feature in [imu.x, imu.y, imu.z, imu.Y, imu.P, imu.R]
    ])
    return audio_window, imu_window

---

## 🎓 ทำความเข้าใจ Features (ฟีเจอร์) ผ่านตัวอย่าง

เราจะนำ **ตัวอย่างจริงสองตัวอย่าง** จากชุดข้อมูลของเรา และดำเนินการตามขั้นตอนการสกัดฟีเจอร์แบบทีละขั้นตอน:

- **ตัวอย่าง A**: **เสียงไอ (cough)** จริงจากชุดข้อมูล
- **ตัวอย่าง B**: **เสียงกระแอม (throat-clearing)** (ไม่ใช่การไอ)

การเปรียบเทียบตัวอย่างเหล่านี้เคียงข้างกัน จะช่วยให้คุณเข้าใจ:
1. ฟีเจอร์ทั้ง 105 ตัวจับอะไรได้บ้าง
2. เหตุใดฟีเจอร์เหล่านี้จึงช่วยแยกแยะการไอกับเสียงอื่น ๆ ได้
3. สัญญาณเสียงและ IMU เสริมกันอย่างไร

แนวคิดที่เป็นนามธรรมของ "ฟีเจอร์" จะเป็นรูปธรรมและเข้าใจง่ายขึ้น ผ่านการปฏิบัตินี้

### ขั้นตอนที่ 0: โหลดชุดข้อมูล (Load dataset)

ก่อนอื่น ให้เริ่มต้นด้วยการเลือก Subject (ผู้ถูกทดลอง) หนึ่งคนจากชุดข้อมูลเพื่อใช้เป็นตัวอย่าง และดำเนินการสกัด Window สำหรับ Subject นั้น

In [ ]:
# Locate dataset folder
kaggle_dataset_dir = '/kaggle/input/edge-ai-cough-count'
base_dir = kaggle_dataset_dir if os.path.exists(kaggle_dataset_dir) else ".."
data_folder = base_dir + '/public_dataset/'

# Check if exists, otherwise try alternative path
if not os.path.exists(data_folder):
    raise FileNotFoundError(
        "Cannot find public_dataset/. Please download from: "
        "https://zenodo.org/record/7562332"
    )

# Get list of subject IDs
subject_ids = [d for d in os.listdir(data_folder) 
               if os.path.isdir(os.path.join(data_folder, d))]
subject_ids = sorted(subject_ids)

print(f"✓ Found {len(subject_ids)} subjects: {subject_ids}")

In [ ]:
# ===================================================================
# Load Example A: Cough Window
# ===================================================================
# Using subject "14287", trial 1, sitting, no noise, cough sound
example_subj = "14287"

# Load full audio signals (both microphones: air-facing and body-facing)
cough_audio_air, cough_audio_skin = load_audio(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.COUGH
)

# Load full IMU signals (all 6 channels: accel x,y,z + gyro Y,P,R)
cough_imu = load_imu(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.COUGH
)

# Load ground truth annotations (start/end times of each cough in seconds)
cough_starts, cough_ends = load_annotation(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.COUGH
)

cough_audio_window, cough_imu_window = extract_subject_window(
    audio=cough_audio_air,
    imu=cough_imu,
    # For window start point, we have ground truth on when the cough happens
    # Let's select the time when we first cough
    start_point_time=cough_starts[0]
)

# ===================================================================
# Load Example B: Throat-Clearing Window (Non-Cough)
# ===================================================================
# Load full throat-clearing signals
throat_audio_air, throat_audio_skin = load_audio(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.THROAT
)
throat_imu = load_imu(
    data_folder, example_subj, Trial.ONE, Movement.SIT, Noise.NONE, Sound.THROAT
)

throat_audio_window, throat_imu_window = extract_subject_window(
    audio=throat_audio_air,
    imu=throat_imu,
    # We don't have a starting point in mind, so pick randomly from the sample length
    start_point_index=np.random.randint(0, len(throat_audio_air))
)

print("✓ Loaded example windows:")
print(f"  Cough: {len(cough_audio_window)} audio samples, {len(cough_imu_window)} IMU samples")
print(f"  Throat-clearing: {len(throat_audio_window)} audio samples, {len(throat_imu_window)} IMU samples")
print(f"\n  Expected: 6400 audio samples (0.4s × 16kHz), 40 IMU samples (0.4s × 100Hz)")

### ขั้นตอนที่ 1: เลือกและแสดงภาพตัวอย่างของ Windows

มาดูสัญญาณดิบ (Raw signals) กัน สังเกตความแตกต่างของรูปแบบคลื่น (waveform patterns) ระหว่างการไอกับเสียงกระแอม

In [ ]:
# Visualize raw signals side-by-side
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Cough audio
time_audio = np.arange(len(cough_audio_window)) / FS_AUDIO
axes[0, 0].plot(time_audio, cough_audio_window, color='steelblue', linewidth=0.8)
axes[0, 0].set_title('Example A: Cough - Audio Signal', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Time (seconds)', fontsize=11)
axes[0, 0].set_ylabel('Amplitude', fontsize=11)
axes[0, 0].grid(alpha=0.3)
# Add annotation for sharp burst
peak_idx = np.argmax(np.abs(cough_audio_window))
peak_val = cough_audio_window[peak_idx]
y_range = axes[0, 0].get_ylim()[1] - axes[0, 0].get_ylim()[0]
text_offset = y_range * 0.20 if peak_val > 0 else -y_range * 0.20
axes[0, 0].annotate('Sharp burst →', 
                    xy=(peak_idx/FS_AUDIO, peak_val),
                    xytext=(peak_idx/FS_AUDIO - 0.08, peak_val + text_offset),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2),
                    fontsize=10, color='red', fontweight='bold')

# Cough IMU (accelerometer Z)
time_imu = np.arange(len(cough_imu_window)) / FS_IMU
axes[0, 1].plot(time_imu, -cough_imu_window[:, 2], color='darkgreen', linewidth=1.5)
axes[0, 1].set_title('Example A: Cough - IMU Accel Z', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Time (seconds)', fontsize=11)
axes[0, 1].set_ylabel('Acceleration', fontsize=11)
axes[0, 1].grid(alpha=0.3)
# Add annotation for IMU spike
imu_peak_idx = np.argmax(-cough_imu_window[:, 2])
imu_peak_val = -cough_imu_window[imu_peak_idx, 2]
y_range_imu = axes[0, 1].get_ylim()[1] - axes[0, 1].get_ylim()[0]
text_offset_imu = y_range_imu * 0.20 if imu_peak_val > 0 else -y_range_imu * 0.20
axes[0, 1].annotate('← IMU spike', 
                    xy=(imu_peak_idx/FS_IMU, imu_peak_val),
                    xytext=(imu_peak_idx/FS_IMU + 0.05, imu_peak_val + text_offset_imu),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2),
                    fontsize=10, color='red', fontweight='bold')

# Throat-clearing audio
axes[1, 0].plot(time_audio, throat_audio_window, color='orange', linewidth=0.8)
axes[1, 0].set_title('Example B: Throat-Clearing - Audio Signal', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Time (seconds)', fontsize=11)
axes[1, 0].set_ylabel('Amplitude', fontsize=11)
axes[1, 0].grid(alpha=0.3)

# Throat-clearing IMU
axes[1, 1].plot(time_imu, -throat_imu_window[:, 2], color='purple', linewidth=1.5)
axes[1, 1].set_title('Example B: Throat-Clearing - IMU Accel Z', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Time (seconds)', fontsize=11)
axes[1, 1].set_ylabel('Acceleration', fontsize=11)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

#### 📊 ข้อสังเกต:
- เสียงไอ (Cough audio): คลื่นเสียงมีความแหลมคม (Sharp) มีการระเบิดอย่างกะทันหัน (sudden burst) และมีความดังของเสียง (amplitude) สูง
- IMU การไอ (Cough IMU): มีการกระชาก (Spike) ที่ชัดเจน สอดคล้องกับการเคลื่อนไหวของหน้าอก/ร่างกาย
- เสียงกระแอม (Throat-clearing): มีรูปแบบเสียงที่ค่อยเป็นค่อยไป (gradual) และต่อเนื่อง (sustained)
- ฟีเจอร์จะจับความแตกต่างเหล่านี้ในเชิงปริมาณ!

### ขั้นตอนที่ 2: ทำความเข้าใจเกี่ยวกับเสียง - Spectrogram

สัญญาณเสียงมีข้อมูลทั้งในด้าน **เวลา (time)** และ **ความถี่ (frequency)** **Spectrogram** จะแสดงให้เห็นว่าความถี่เปลี่ยนแปลงไปตามเวลาอย่างไร คล้ายกับโน้ตดนตรีที่แสดงให้เห็นว่าโน้ตใด (ความถี่ใด) ถูกเล่นเมื่อใด

**ลองคิดแบบนี้:**
- **ความถี่ต่ำ (Low frequencies)** (ด้านล่างของ spectrogram): เสียงทุ้มลึก, เสียงเบส
- **ความถี่สูง (High frequencies)** (ด้านบนของ spectrogram): เสียงแหลมคม, เสียงสูง เช่น เสียง "ส" หรือ "ช"
- **ความเข้มของสี (Color intensity)**: ความดังของความถี่นั้นในขณะนั้น

มาแสดงภาพ Spectrogram สำหรับตัวอย่างการไอกับเสียงกระแอมของเรา:

In [ ]:
# Create spectrograms
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cough spectrogram
D_cough = librosa.amplitude_to_db(np.abs(librosa.stft(cough_audio_window)), ref=np.max)
img1 = librosa.display.specshow(D_cough, sr=FS_AUDIO, x_axis='time', y_axis='hz', 
                                 ax=axes[0], cmap='viridis')
axes[0].set_title('Example A: Cough - Spectrogram', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Time (seconds)', fontsize=11)
axes[0].set_ylabel('Frequency (Hz)', fontsize=11)
axes[0].set_ylim(0, 8000)
fig.colorbar(img1, ax=axes[0], format='%+2.0f dB')

# Throat-clearing spectrogram
D_throat = librosa.amplitude_to_db(np.abs(librosa.stft(throat_audio_window)), ref=np.max)
img2 = librosa.display.specshow(D_throat, sr=FS_AUDIO, x_axis='time', y_axis='hz',
                                 ax=axes[1], cmap='viridis')
axes[1].set_title('Example B: Throat-Clearing - Spectrogram', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Time (seconds)', fontsize=11)
axes[1].set_ylabel('Frequency (Hz)', fontsize=11)
axes[1].set_ylim(0, 8000)
fig.colorbar(img2, ax=axes[1], format='%+2.0f dB')

plt.tight_layout()
plt.show()

#### 📊 สิ่งที่เราสังเกต:
- การไอ (Cough): แถบแนวตั้งที่คมชัด (sudden burst) พร้อมเนื้อหาความถี่สูงที่รุนแรง (สีเหลืองสดใสที่ด้านบน)
- เสียงกระแอม (Throat-clearing): มีการกระจายในแนวนอนมากกว่า (sustained) โดยมีพลังงานกระจุกตัวอยู่ในความถี่ที่ต่ำกว่า
- เสียงไอเหมือนกับการตีฉาบ – มีความถี่สูงจำนวนมาก
- เสียงกระแอมเป็นเหมือนการเคลียร์สายเสียง – ต่ำกว่าและค่อยเป็นค่อยไปมากกว่า

### ขั้นตอนที่ 3: MFCC Features - การจับ "พื้นผิว (Texture)" ของเสียง

**MFCC** ย่อมาจาก **Mel-Frequency Cepstral Coefficients (สัมประสิทธิ์เซปสตรัมบนสเกลเมล)** - เป็นชื่อที่ดูหรูหราสำหรับฟีเจอร์ที่จับ "รูปร่าง" หรือ "พื้นผิว" ของเสียง

- MFCC ได้รับแรงบันดาลใจจากวิธีที่หูของมนุษย์รับรู้เสียง
- สเกล "Mel" เลียนแบบวิธีที่เราได้ยิน: เราแยกแยะความถี่ต่ำได้ดีกว่าความถี่สูง
- เราสกัด **13 ค่า MFCC** ณ จุดเวลาแต่ละจุด จากนั้นคำนวณสถิติ (ค่าเฉลี่ย, ค่าเบี่ยงเบนมาตรฐาน, ค่าต่ำสุด, ค่าสูงสุด) ตลอด Window
- สิ่งนี้ทำให้เราได้ **13 × 4 = 52 ฟีเจอร์ MFCC** ต่อ Audio Window

**เหตุผลที่ MFCC มีความสำคัญ:**
- MFCC Cooefficients 1-3 จับรูปร่าง Spectrum โดยรวม (การไอจะคมชัดและเป็นการกระทบมากกว่า)
- Coefficient ที่สูงกว่าจะจับรายละเอียดปลีกย่อยของพื้นผิวเสียง
- ถูกใช้กันอย่างแพร่หลายในการรู้จำคำพูด (speech recognition) และการจัดประเภทเสียง (audio classification)

In [ ]:
# Compute MFCCs for both examples
mfcc_cough = librosa.feature.mfcc(y=cough_audio_window, sr=FS_AUDIO, n_mfcc=13)
mfcc_throat = librosa.feature.mfcc(y=throat_audio_window, sr=FS_AUDIO, n_mfcc=13)

# Compute statistics for comparison
mfcc_cough_mean = np.mean(mfcc_cough, axis=1)
mfcc_throat_mean = np.mean(mfcc_throat, axis=1)
mfcc_cough_std = np.std(mfcc_cough, axis=1)
mfcc_throat_std = np.std(mfcc_throat, axis=1)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# MFCC heatmap for cough
img1 = axes[0, 0].imshow(mfcc_cough, aspect='auto', origin='lower', cmap='coolwarm')
axes[0, 0].set_title('Example A: Cough - MFCC Heatmap', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Time Frame', fontsize=11)
axes[0, 0].set_ylabel('MFCC Coefficient', fontsize=11)
axes[0, 0].set_yticks(range(13))
fig.colorbar(img1, ax=axes[0, 0])

# MFCC heatmap for throat-clearing
img2 = axes[0, 1].imshow(mfcc_throat, aspect='auto', origin='lower', cmap='coolwarm')
axes[0, 1].set_title('Example B: Throat-Clearing - MFCC Heatmap', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Time Frame', fontsize=11)
axes[0, 1].set_ylabel('MFCC Coefficient', fontsize=11)
axes[0, 1].set_yticks(range(13))
fig.colorbar(img2, ax=axes[0, 1])

# Bar chart of MFCC means
x = np.arange(13)
width = 0.35
axes[1, 0].bar(x - width/2, mfcc_cough_mean, width, label='Cough', color='steelblue', alpha=0.8)
axes[1, 0].bar(x + width/2, mfcc_throat_mean, width, label='Throat-clearing', color='orange', alpha=0.8)
axes[1, 0].set_title('MFCC Mean Comparison', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('MFCC Coefficient', fontsize=11)
axes[1, 0].set_ylabel('Mean Value', fontsize=11)
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3, axis='y')

# Bar chart of MFCC standard deviations
axes[1, 1].bar(x - width/2, mfcc_cough_std, width, label='Cough', color='steelblue', alpha=0.8)
axes[1, 1].bar(x + width/2, mfcc_throat_std, width, label='Throat-clearing', color='orange', alpha=0.8)
axes[1, 1].set_title('MFCC Standard Deviation Comparison', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('MFCC Coefficient', fontsize=11)
axes[1, 1].set_ylabel('Std Value', fontsize=11)
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

#### 📊 ข้อมูลเชิงลึกที่สำคัญ:
- สังเกตว่า Coefficient 0-3 แตกต่างกันอย่างมากระหว่างการไอและการกระแอม
- สิ่งเหล่านี้จับรูปร่าง Spectrum โดยรวม – การไอจะมีความคมชัดและเป็นการกระทบมากกว่า
- ค่าเบี่ยงเบนมาตรฐาน (ความแปรปรวนเมื่อเวลาผ่านไป) ก็แตกต่างกันด้วย
- เมื่อรวมกันแล้ว ฟีเจอร์ 52 ตัวเหล่านี้ (13 Coefficient × 4 สถิติ) ให้ลายเซ็น (Signature) ที่มีประสิทธิภาพ!

### ขั้นตอนที่ 4: Spectral Features - ลักษณะความถี่ (Frequency Characteristics)

Spectral features อธิบายว่า **พลังงานกระจุกตัวอยู่ที่ใด** ใน Spectrum ความถี่ ลองนึกภาพว่าฟีเจอร์เหล่านี้เป็นการอธิบาย "สี" ของเสียง

- **Spectral Centroid**: "จุดศูนย์กลางมวล" ของความถี่ - เช่น การหาความถี่เฉลี่ย โดย Centroid ที่สูงหมายถึงมีเนื้อหาความถี่สูงมากขึ้น (เช่น เสียง "ซ")
- **Spectral Rolloff**: ความถี่ที่มีพลังงานต่ำกว่า 85% กระจุกตัวอยู่ โดย Rolloff ที่สูงขึ้น = พลังงานความถี่สูงมากขึ้น
- **Spectral Bandwidth**: "การกระจาย" ของความถี่ - ช่วงความถี่กว้างแค่ไหน
- **Spectral Flatness**: เสียงมีความ "เป็นสัญญาณรบกวน (noisy)" หรือ "เป็นโทนเสียง (tonal)" มากน้อยเพียงใด (White noise = 1, Pure tone = 0)

มาแสดงภาพสิ่งเหล่านี้สำหรับตัวอย่างของเรา:

In [ ]:
# Compute spectral features

# Compute PSD (Power Spectral Density)
freqs_cough, psd_cough = signal.welch(cough_audio_window, fs=FS_AUDIO, nperseg=512)
freqs_throat, psd_throat = signal.welch(throat_audio_window, fs=FS_AUDIO, nperseg=512)

# Compute spectral centroid
centroid_cough = np.mean(librosa.feature.spectral_centroid(y=cough_audio_window, sr=FS_AUDIO))
centroid_throat = np.mean(librosa.feature.spectral_centroid(y=throat_audio_window, sr=FS_AUDIO))

# Compute spectral rolloff
rolloff_cough = np.mean(librosa.feature.spectral_rolloff(y=cough_audio_window, sr=FS_AUDIO))
rolloff_throat = np.mean(librosa.feature.spectral_rolloff(y=throat_audio_window, sr=FS_AUDIO))

# Compute spectral bandwidth
bandwidth_cough = np.mean(librosa.feature.spectral_bandwidth(y=cough_audio_window, sr=FS_AUDIO))
bandwidth_throat = np.mean(librosa.feature.spectral_bandwidth(y=throat_audio_window, sr=FS_AUDIO))

# Visualize PSD with spectral features
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cough PSD
axes[0].plot(freqs_cough, psd_cough, color='steelblue', linewidth=2, label='PSD')
axes[0].axvline(centroid_cough, color='red', linestyle='--', linewidth=2, label=f'Centroid: {centroid_cough:.0f} Hz')
axes[0].axvline(rolloff_cough, color='blue', linestyle='--', linewidth=2, label=f'Rolloff: {rolloff_cough:.0f} Hz')
axes[0].axvspan(centroid_cough - bandwidth_cough/2, centroid_cough + bandwidth_cough/2, 
                alpha=0.2, color='green', label=f'Bandwidth: {bandwidth_cough:.0f} Hz')
axes[0].set_title('Example A: Cough - Spectral Features', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Frequency (Hz)', fontsize=11)
axes[0].set_ylabel('Power Spectral Density', fontsize=11)
axes[0].set_xlim(0, 8000)
axes[0].legend(fontsize=9)
axes[0].grid(alpha=0.3)

# Throat-clearing PSD
axes[1].plot(freqs_throat, psd_throat, color='orange', linewidth=2, label='PSD')
axes[1].axvline(centroid_throat, color='red', linestyle='--', linewidth=2, label=f'Centroid: {centroid_throat:.0f} Hz')
axes[1].axvline(rolloff_throat, color='blue', linestyle='--', linewidth=2, label=f'Rolloff: {rolloff_throat:.0f} Hz')
axes[1].axvspan(centroid_throat - bandwidth_throat/2, centroid_throat + bandwidth_throat/2,
                alpha=0.2, color='green', label=f'Bandwidth: {bandwidth_throat:.0f} Hz')
axes[1].set_title('Example B: Throat-Clearing - Spectral Features', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Frequency (Hz)', fontsize=11)
axes[1].set_ylabel('Power Spectral Density', fontsize=11)
axes[1].set_xlim(0, 8000)
axes[1].legend(fontsize=9)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Create comparison table
print("\n📊 Spectral Feature Comparison:")
print(f"{'Feature':<20} {'Cough':>15} {'Throat-Clearing':>20}")
print("-" * 60)
print(f"{'Spectral Centroid':<20} {centroid_cough:>12.1f} Hz {centroid_throat:>17.1f} Hz")
print(f"{'Spectral Rolloff':<20} {rolloff_cough:>12.1f} Hz {rolloff_throat:>17.1f} Hz")
print(f"{'Spectral Bandwidth':<20} {bandwidth_cough:>12.1f} Hz {bandwidth_throat:>17.1f} Hz")

#### 💡 ข้อคิดเห็น:
- โดยทั่วไปแล้ว การไอจะมี Spectral Centroid และ Rolloff ที่สูงกว่า (มีเสียง 'ฟู่' ของความถี่สูงมากกว่า)
- เสียงกระแอมมีพลังงานในความถี่ที่ต่ำกว่า (การสั่นของสายเสียง)
- ความแตกต่างเหล่านี้ช่วยให้โมเดล ML แยกแยะการไอกับเสียงอื่น ๆ ได้!

### ขั้นตอนที่ 5: Time-Domain Features - ลักษณะเชิงเวลา (Temporal Characteristics)

Time-domain features อธิบายคุณสมบัติของรูปแบบคลื่น **ตามเวลา** (แทนที่จะเป็นความถี่)

- **Zero-Crossing Rate (ZCR)**: ความถี่ที่สัญญาณข้ามศูนย์ (เกี่ยวข้องกับความถี่) โดย ZCR สูง = มีการแกว่ง (Oscillations) จำนวนมาก
- **RMS Energy**: Root Mean Square Energy – การวัด "ความดัง" หรือพลังงานโดยรวม
- **Crest Factor**: อัตราส่วนของแอมพลิจูดสูงสุด (peak amplitude) ต่อ RMS โดยวัดว่าสัญญาณมีความ "เป็นยอด (peaky)" หรือ "เป็นหนามแหลม (spiky)" เพียงใด
  - Crest Factor สูง = สัญญาณชั่วคราวที่คมชัด (เช่น การไอ)
  - Crest Factor ต่ำ = เสียงที่ราบรื่นและต่อเนื่อง (เช่น เสียงสระ)

---

**⚠️ ข้อควรทราบที่สำคัญเกี่ยวกับ DC Offset และการ Preprocess:**

ตามวิธีของงานวิจัย:
- **สัญญาณ IMU**: ค่าเฉลี่ยของ DC Offset จะถูกลบออกอย่างชัดเจนจาก Window 0.4 วินาทีแต่ละ Window ก่อนการสกัดฟีเจอร์
- **สัญญาณเสียง (Audio signals)**: ไม่ได้มีการกล่าวถึงการลบ DC Offset โดยฟีเจอร์จะถูกคำนวณจากเสียงดิบ (raw audio)

**สำหรับโค้ด Visualization ในเซลล์ด้านล่างนี้เท่านั้น** เราจะลบ DC Offset ออกจากเสียงเพื่อปรับปรุงให้ภาพชัดเจนจะขึ้น เมื่อเปรียบเทียบระดับ Peak/RMS สิ่งนี้ทำให้มองเห็นส่วนประกอบ AC (การแกว่งที่เราสนใจ) ได้ง่ายขึ้น

**ฟังก์ชันการสกัดฟีเจอร์จริงจะคำนวณฟีเจอร์บนสัญญาณเสียงดิบ** (โดยที่ DC Offset ยังคงอยู่) ซึ่งเป็นไปตามเอกสารต้นฉบับ การแสดงภาพด้านล่างนี้มีไว้เพื่อวัตถุประสงค์ทางการศึกษาเท่านั้น

มาแสดงภาพสิ่งเหล่านี้:

In [ ]:
# Compute time-domain features
# Zero-crossing rate
zcr_cough = librosa.feature.zero_crossing_rate(cough_audio_window)[0].mean()
zcr_throat = librosa.feature.zero_crossing_rate(throat_audio_window)[0].mean()

# RMS energy
rms_cough = np.sqrt(np.mean(cough_audio_window**2))
rms_throat = np.sqrt(np.mean(throat_audio_window**2))

# Crest factor
crest_cough = np.max(np.abs(cough_audio_window)) / (rms_cough + 1e-10)
crest_throat = np.max(np.abs(throat_audio_window)) / (rms_throat + 1e-10)

# ===================================================================
# For VISUALIZATION ONLY: Remove DC offset for clarity
# ===================================================================
cough_centered = cough_audio_window - np.mean(cough_audio_window)
throat_centered = throat_audio_window - np.mean(throat_audio_window)

# Compute visualization-only metrics on DC-centered signals
zcr_cough_vis = np.sum(np.diff(np.sign(cough_centered)) != 0) / len(cough_centered)
zcr_throat_vis = np.sum(np.diff(np.sign(throat_centered)) != 0) / len(throat_centered)

rms_cough_vis = np.sqrt(np.mean(cough_centered**2))
rms_throat_vis = np.sqrt(np.mean(throat_centered**2))

peak_cough_vis = np.max(np.abs(cough_centered))
peak_throat_vis = np.max(np.abs(throat_centered))

crest_cough_vis = peak_cough_vis / (rms_cough_vis + 1e-10)
crest_throat_vis = peak_throat_vis / (rms_throat_vis + 1e-10)

# ===================================================================
# Visualize
# ===================================================================
fig, axes = plt.subplots(3, 2, figsize=(14, 12))

# Row 1: Waveforms with zero crossings (DC-centered)
time_audio = np.arange(len(cough_audio_window)) / FS_AUDIO

axes[0, 0].plot(time_audio, cough_centered, color='steelblue', linewidth=0.8, label='Waveform (DC removed)')
axes[0, 0].axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
zero_crossings_cough = np.where(np.diff(np.sign(cough_centered)))[0]
axes[0, 0].scatter(zero_crossings_cough[:100]/FS_AUDIO, 
                   np.zeros(min(100, len(zero_crossings_cough))), 
                   color='red', s=10, alpha=0.5, label='Zero crossings (first 100)')
axes[0, 0].set_title(f'Example A: Cough - Zero Crossings (ZCR={zcr_cough_vis:.4f})', 
                     fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Time (seconds)', fontsize=10)
axes[0, 0].set_ylabel('Amplitude (DC removed)', fontsize=10)
axes[0, 0].legend(fontsize=9)
axes[0, 0].grid(alpha=0.3)

axes[0, 1].plot(time_audio, throat_centered, color='orange', linewidth=0.8, label='Waveform (DC removed)')
axes[0, 1].axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
zero_crossings_throat = np.where(np.diff(np.sign(throat_centered)))[0]
axes[0, 1].scatter(zero_crossings_throat[:100]/FS_AUDIO,
                   np.zeros(min(100, len(zero_crossings_throat))),
                   color='red', s=10, alpha=0.5, label='Zero crossings (first 100)')
axes[0, 1].set_title(f'Example B: Throat-Clearing - Zero Crossings (ZCR={zcr_throat_vis:.4f})',
                     fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Time (seconds)', fontsize=10)
axes[0, 1].set_ylabel('Amplitude (DC removed)', fontsize=10)
axes[0, 1].legend(fontsize=9)
axes[0, 1].grid(alpha=0.3)

# Row 2: RMS Energy comparison (visualization values)
axes[1, 0].bar(['Cough', 'Throat-Clearing'], [rms_cough_vis, rms_throat_vis], 
               color=['steelblue', 'orange'], alpha=0.7)
axes[1, 0].set_title('RMS Energy Comparison (DC removed)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('RMS Energy', fontsize=10)
axes[1, 0].grid(alpha=0.3, axis='y')
for i, (label, val) in enumerate([('Cough', rms_cough_vis), ('Throat', rms_throat_vis)]):
    axes[1, 0].text(i, val, f'{val:.4f}', ha='center', va='bottom', fontsize=10)

# Row 2: Crest Factor comparison (visualization values)
axes[1, 1].bar(['Cough', 'Throat-Clearing'], [crest_cough_vis, crest_throat_vis],
               color=['steelblue', 'orange'], alpha=0.7)
axes[1, 1].set_title('Crest Factor Comparison (DC removed)', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Crest Factor (Peak/RMS)', fontsize=10)
axes[1, 1].grid(alpha=0.3, axis='y')
for i, (label, val) in enumerate([('Cough', crest_cough_vis), ('Throat', crest_throat_vis)]):
    axes[1, 1].text(i, val, f'{val:.2f}', ha='center', va='bottom', fontsize=10)

# Row 3: Waveforms with peak and RMS lines (DC-centered)
axes[2, 0].plot(time_audio, cough_centered, color='steelblue', linewidth=0.8, label='Waveform (DC removed)')
axes[2, 0].axhline(rms_cough_vis, color='green', linestyle='--', linewidth=2, label=f'RMS: {rms_cough_vis:.4f}')
axes[2, 0].axhline(-rms_cough_vis, color='green', linestyle='--', linewidth=2)
axes[2, 0].axhline(peak_cough_vis, color='red', linestyle='--', linewidth=2, label=f'Peak: {peak_cough_vis:.4f}')
axes[2, 0].axhline(-peak_cough_vis, color='red', linestyle='--', linewidth=2)
axes[2, 0].set_title(f'Example A: Cough - Peak vs RMS (Crest={crest_cough_vis:.2f})',
                     fontsize=12, fontweight='bold')
axes[2, 0].set_xlabel('Time (seconds)', fontsize=10)
axes[2, 0].set_ylabel('Amplitude (DC removed)', fontsize=10)
axes[2, 0].legend(fontsize=9)
axes[2, 0].grid(alpha=0.3)

axes[2, 1].plot(time_audio, throat_centered, color='orange', linewidth=0.8, label='Waveform (DC removed)')
axes[2, 1].axhline(rms_throat_vis, color='green', linestyle='--', linewidth=2, label=f'RMS: {rms_throat_vis:.4f}')
axes[2, 1].axhline(-rms_throat_vis, color='green', linestyle='--', linewidth=2)
axes[2, 1].axhline(peak_throat_vis, color='red', linestyle='--', linewidth=2, label=f'Peak: {peak_throat_vis:.4f}')
axes[2, 1].axhline(-peak_throat_vis, color='red', linestyle='--', linewidth=2)
axes[2, 1].set_title(f'Example B: Throat-Clearing - Peak vs RMS (Crest={crest_throat_vis:.2f})',
                     fontsize=12, fontweight='bold')
axes[2, 1].set_xlabel('Time (seconds)', fontsize=10)
axes[2, 1].set_ylabel('Amplitude (DC removed)', fontsize=10)
axes[2, 1].legend(fontsize=9)
axes[2, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# ===================================================================
# Print comparison table using ACTUAL features (raw signal)
# ===================================================================
print("\n📊 Time-Domain Feature Comparison (ACTUAL features used by ML model):")
print(f"{'Feature':<20} {'Cough':>15} {'Throat-Clearing':>20}")
print("-" * 60)
print(f"{'Zero-Crossing Rate':<20} {zcr_cough:>15.4f} {zcr_throat:>20.4f}")
print(f"{'RMS Energy':<20} {rms_cough:>15.4f} {rms_throat:>20.4f}")
print(f"{'Crest Factor':<20} {crest_cough:>15.2f} {crest_throat:>20.2f}")
print(f"{'DC Offset (mean)':<20} {np.mean(cough_audio_window):>15.4f} {np.mean(throat_audio_window):>20.4f}")

print("\n📊 Visualization-Only Metrics (DC-centered for clarity):")
print(f"{'Feature':<20} {'Cough':>15} {'Throat-Clearing':>20}")
print("-" * 60)
print(f"{'Zero-Crossing Rate':<20} {zcr_cough_vis:>15.4f} {zcr_throat_vis:>20.4f}")
print(f"{'RMS Energy':<20} {rms_cough_vis:>15.4f} {rms_throat_vis:>20.4f}")
print(f"{'Crest Factor':<20} {crest_cough_vis:>15.2f} {crest_throat_vis:>20.2f}")

#### 💡 ข้อคิดเห็น:
- การไอมี Crest Factor สูง (ยอดที่แหลมคม) – เป็นลักษณะเฉพาะของเสียงที่กระแทกกระทั้น (impulsive sounds)
- เสียงกระแอมมี Crest Factor ต่ำกว่า – ค่อยเป็นค่อยไปและต่อเนื่องมากกว่า
- ZCR ที่สูงขึ้นในการไอ สะท้อนถึงลักษณะของเสียงที่เป็นสัญญาณรบกวน (noisy, broadband nature)

#### ⚠️ หมายเหตุ:
- ฟีเจอร์ ML ที่แท้จริงถูกคำนวณจากเสียงดิบ (พร้อม DC Offset)
- การแสดงภาพใช้เสียงที่อยู่ตรงกลาง DC เพื่อความชัดเจนเท่านั้น
- ซึ่งเป็นไปตามระเบียบวิธีของเอกสาร: ไม่มีการลบ DC สำหรับเสียง แต่มีการลบ DC สำหรับ IMU

### ขั้นตอนที่ 6: IMU Features - สัญญาณการเคลื่อนไหว (Motion Signals)

ตอนนี้มาดูที่ **Inertial Measurement Unit (IMU)** ซึ่งเป็นเซ็นเซอร์ตรวจจับการเคลื่อนไหว

- **IMU**: เซ็นเซอร์ที่วัดการเคลื่อนไหวโดยใช้ส่วนประกอบสองส่วน:
  - **Accelerometer**: วัดความเร่งเชิงเส้น (Linear acceleration) (เช่น ความรู้สึกถูกผลักในรถ) จับการเคลื่อนไหวของหน้าอก/ร่างกายระหว่างการไอ
  - **Gyroscope**: วัดการหมุน (Rotation) (เช่น การหันศีรษะ) จับการเคลื่อนไหวของศีรษะ/คอระหว่างการไอ

- **เซ็นเซอร์แต่ละตัวมี 3 แกน**: X, Y, Z (วัดการเคลื่อนไหวในพื้นที่ 3 มิติ)
- **L2 norm**: ขนาดการเคลื่อนไหวทั้งหมด คำนวณเป็น √(x² + y² + z²)
  - ลองนึกถึงการคำนวณระยะทาง: ถ้าคุณเดินไปทางเหนือ 3 เมตร และไปทางตะวันออก 4 เมตร คุณจะอยู่ห่างจากจุดเริ่มต้นเป็นระยะทาง √(3² + 4²) = 5 เมตร

**รวม**: 3 accel + 1 accel_L2 + 3 gyro + 1 gyro_L2 = **8 สัญญาณ** × 5 ฟีเจอร์ต่อสัญญาณ = **40 IMU features**

มาแสดงภาพสัญญาณทั้ง 8:

In [ ]:
# Compute L2 norms
# First, center the signals (subtract mean) as per the paper
cough_imu_centered = cough_imu_window - np.mean(cough_imu_window, axis=0, keepdims=True)
throat_imu_centered = throat_imu_window - np.mean(throat_imu_window, axis=0, keepdims=True)

# Compute L2 norms
cough_accel_l2 = np.linalg.norm(cough_imu_centered[:, 0:3], axis=1)
cough_gyro_l2 = np.linalg.norm(cough_imu_centered[:, 3:6], axis=1)
throat_accel_l2 = np.linalg.norm(throat_imu_centered[:, 0:3], axis=1)
throat_gyro_l2 = np.linalg.norm(throat_imu_centered[:, 3:6], axis=1)

# Build per-feature signal arrays (same order for both classes)
time_imu = np.arange(40) / FS_IMU
signal_names = ['Accel X', 'Accel Y', 'Accel Z', 'Accel L2', 'Gyro Y', 'Gyro P', 'Gyro R', 'Gyro L2']
colors = ['steelblue', 'darkgreen', 'purple', 'red', 'orange', 'brown', 'pink', 'darkred']

cough_signals = [
    cough_imu_centered[:, 0],  # Accel X
    cough_imu_centered[:, 1],  # Accel Y
    cough_imu_centered[:, 2],  # Accel Z
    cough_accel_l2,            # Accel L2
    cough_imu_centered[:, 3],  # Gyro Y
    cough_imu_centered[:, 4],  # Gyro P
    cough_imu_centered[:, 5],  # Gyro R
    cough_gyro_l2              # Gyro L2
]

throat_signals = [
    throat_imu_centered[:, 0],  # Accel X
    throat_imu_centered[:, 1],  # Accel Y
    throat_imu_centered[:, 2],  # Accel Z
    throat_accel_l2,            # Accel L2
    throat_imu_centered[:, 3],  # Gyro Y
    throat_imu_centered[:, 4],  # Gyro P
    throat_imu_centered[:, 5],  # Gyro R
    throat_gyro_l2              # Gyro L2
]

# --- One figure: 8 rows (features) × 2 cols (Cough vs Throat) ---
fig, axes = plt.subplots(
    nrows=8, ncols=2,
    figsize=(16, 18),
    sharex=True
)

for i, (name, color) in enumerate(zip(signal_names, colors)):
    # Left column: Cough
    axL = axes[i, 0]
    axL.plot(time_imu, cough_signals[i], color=color, linewidth=2)
    axL.set_title(f'Cough - {name}', fontsize=11, fontweight='bold')
    axL.set_ylabel('Magnitude', fontsize=9)
    axL.grid(alpha=0.3)
    axL.axhline(0, color='black', linestyle='--', linewidth=0.5, alpha=0.5)

    # Right column: Throat-clearing
    axR = axes[i, 1]
    axR.plot(time_imu, throat_signals[i], color=color, linewidth=2)
    axR.set_title(f'Throat-Clearing - {name}', fontsize=11, fontweight='bold')
    axR.grid(alpha=0.3)
    axR.axhline(0, color='black', linestyle='--', linewidth=0.5, alpha=0.5)

# Only bottom row gets x-labels to reduce clutter
axes[-1, 0].set_xlabel('Time (seconds)', fontsize=9)
axes[-1, 1].set_xlabel('Time (seconds)', fontsize=9)

plt.suptitle('IMU Features - Motion Signals (Cough vs Throat-Clearing)', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

#### 📊 ข้อสังเกต:
- การไอสร้างรูปแบบการเคลื่อนไหวที่แตกต่างกัน:
    - การเคลื่อนไหวของหน้าอกอย่างกะทันหัน (accelerometer)
    - การเคลื่อนไหวของศีรษะ/คอ (gyroscope)
- L2 norms จับขนาดรวมทั้งหมดของการเคลื่อนไหว โดยไม่คำนึงถึงทิศทาง
- ลายเซ็นการเคลื่อนไหว (motion signatures) เหล่านี้ช่วยเสริมฟีเจอร์เสียงสำหรับการตรวจจับที่แข็งแกร่ง

### ขั้นตอนที่ 7: IMU Statistical Features

สำหรับสัญญาณ IMU ทั้ง 8 สัญญาณ เราจะคำนวณ **5 ฟีเจอร์ทางสถิติ**:

- **Line Length**: ผลรวมของความแตกต่างสัมบูรณ์ระหว่าง Samples ที่อยู่ติดกัน วัดว่าสัญญาณมีความ "กระดิก (wiggly)" หรือ active เพียงใด
- **Zero-Crossing Rate**: ความถี่ที่สัญญาณเปลี่ยนเครื่องหมาย (ข้ามศูนย์) คล้ายกับ Audio ZCR
- **Kurtosis**: การวัด "ความแหลมคม (peakiness)" โดย Kurtosis สูง = สัญญาณมี Spike ที่แหลมคม; Kurtosis ต่ำ = สัญญาณที่ราบรื่น
- **Crest Factor**: อัตราส่วน Peak-to-RMS (เหมือนกับเสียง) วัดความกระแทกกระทั้น (impulsiveness)
- **RMS**: Root Mean Square - ขนาดโดยรวมของสัญญาณ

มาสกัดฟีเจอร์ทั้ง 40 ตัวและเปรียบเทียบการไอกับเสียงกระแอม:

In [ ]:
# Extract full 40 IMU features using our feature extraction function
imu_features_cough = extract_imu_features(cough_imu_window)
imu_features_throat = extract_imu_features(throat_imu_window)

# Create feature names for visualization
feature_types = ['Line Length', 'ZCR', 'Kurtosis', 'Crest', 'RMS']
signal_names_short = ['Acc_X', 'Acc_Y', 'Acc_Z', 'Acc_L2', 'Gyro_Y', 'Gyro_P', 'Gyro_R', 'Gyro_L2']
feature_names = []
for sig in signal_names_short:
    for feat in feature_types:
        feature_names.append(f'{sig}_{feat}')

# Visualize all 40 features as a bar chart
fig, ax = plt.subplots(figsize=(16, 8))
x = np.arange(40)
width = 0.35

# Use raw feature values (no normalization for 2-sample comparison)
ax.bar(x - width/2, imu_features_cough, width, label='Cough', color='steelblue', alpha=0.7)
ax.bar(x + width/2, imu_features_throat, width, label='Throat-Clearing', color='orange', alpha=0.7)

ax.set_title('All 40 IMU Features Comparison (Raw Values)', fontsize=14, fontweight='bold')
ax.set_xlabel('Feature Index', fontsize=12)
ax.set_ylabel('Feature Value', fontsize=12)
ax.set_xticks(x[::5])  # Show every 5th tick
ax.set_xticklabels(range(0, 40, 5))
ax.legend(fontsize=11)
ax.grid(alpha=0.3, axis='y')
ax.axhline(0, color='black', linestyle='-', linewidth=0.5)

# Add vertical lines to separate signal groups
for i in range(1, 8):
    ax.axvline(i * 5 - 0.5, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)

# Add signal labels
signal_positions = [2.5, 7.5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5]
for pos, sig_name in zip(signal_positions, signal_names_short):
    ax.text(pos, ax.get_ylim()[1] * 0.9, sig_name, ha='center', fontsize=8,
            rotation=0, fontweight='bold', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.tight_layout()
plt.show()

# Find top 10 most discriminative features (largest absolute difference)
differences = np.abs(imu_features_cough - imu_features_throat)
top_indices = np.argsort(differences)[-10:][::-1]

print("\n📊 Top 10 Most Discriminative IMU Features:")
print(f"{'Rank':<6} {'Feature Name':<25} {'Cough':>12} {'Throat':>12} {'Diff':>12}")
print("-" * 75)
for rank, idx in enumerate(top_indices, 1):
    print(f"{rank:<6} {feature_names[idx]:<25} {imu_features_cough[idx]:>12.4f} {imu_features_throat[idx]:>12.4f} {differences[idx]:>12.4f}")

#### 💡 ข้อมูลเชิงลึกที่สำคัญ:

- **Gyroscope Line Length มีอิทธิพลเหนือกว่า** - ฟีเจอร์ 5 อันดับแรกทั้งหมดวัดความเข้มของการเคลื่อนไหวแบบหมุน (rotational motion)
- การไอทำให้เกิด**กิจกรรม Gyroscope มากกว่า** เสียงกระแอมถึง 3 เท่า (การหมุนศีรษะ/คอ)
- Line Length (ความ 'กระดิก' ของสัญญาณ) มีความสามารถในการแยกแยะมากกว่าการวัดทางสถิติ
- Gyro_R (Roll) แสดงความแตกต่างที่มากที่สุด - จับการเอียงศีรษะระหว่างการไอ
- IMU จับกลไกทางชีวภาพ: การไอ = การเคลื่อนไหวของศีรษะ/คออย่างกะทันหัน, เสียงกระแอม = การเคลื่อนไหวที่นิ่งกว่า
- ฟีเจอร์ 40 ตัวนี้เสริมฟีเจอร์เสียงสำหรับการตรวจจับแบบ Multimodal ที่แข็งแกร่ง!

### ขั้นตอนที่ 8: Vector ฟีเจอร์ที่สมบูรณ์ - การรวบรวมทั้งหมด

ตอนนี้มาสกัด **Vector ทั้ง 105 ฟีเจอร์ที่สมบูรณ์** (65 เสียง + 40 IMU) สำหรับทั้งสองตัวอย่าง และแสดงภาพผลลัพธ์สุดท้าย

นี่คือสิ่งที่ป้อนเข้าสู่โมเดล Machine Learning!

In [ ]:
# Extract complete feature vectors
audio_features_cough = extract_audio_features(cough_audio_window, fs=FS_AUDIO)
audio_features_throat = extract_audio_features(throat_audio_window, fs=FS_AUDIO)

# Combine audio + IMU
complete_features_cough = np.concatenate([audio_features_cough, imu_features_cough])
complete_features_throat = np.concatenate([audio_features_throat, imu_features_throat])

print(f"✓ Complete feature vectors extracted:")
print(f"  Cough: {len(complete_features_cough)} features")
print(f"  Throat-clearing: {len(complete_features_throat)} features")
print(f"\n  Breakdown: 65 audio + 40 IMU = 105 total\n")

# Visualize as grouped bar chart
fig, axes = plt.subplots(3, 1, figsize=(16, 14))

# Use log scale for better visualization of features with different magnitudes
# Apply log(1 + abs(x)) * sign(x) transformation to preserve sign
def log_transform(x):
    return np.sign(x) * np.log1p(np.abs(x))

cough_log = log_transform(complete_features_cough)
throat_log = log_transform(complete_features_throat)

# Panel 1: All 105 features
x = np.arange(105)
width = 0.35
axes[0].bar(x - width/2, cough_log, width, label='Cough', color='steelblue', alpha=0.7)
axes[0].bar(x + width/2, throat_log, width, label='Throat-Clearing', color='orange', alpha=0.7)
axes[0].set_title('Complete 105-Feature Vector (Log-Scaled)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature Index', fontsize=12)
axes[0].set_ylabel('Log-Scaled Feature Value', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3, axis='y')
axes[0].axhline(0, color='black', linestyle='-', linewidth=0.5)
# Add divider between audio and IMU
axes[0].axvline(64.5, color='red', linestyle='--', linewidth=2, label='Audio | IMU')
axes[0].text(32, axes[0].get_ylim()[1] * 0.85, 'Audio Features (65)', 
             ha='center', fontsize=11, fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
axes[0].text(82, axes[0].get_ylim()[1] * 0.85, 'IMU Features (40)',
             ha='center', fontsize=11, fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

# Panel 2: Audio features only (first 65)
x_audio = np.arange(65)
axes[1].bar(x_audio - width/2, cough_log[:65], width, label='Cough', color='steelblue', alpha=0.7)
axes[1].bar(x_audio + width/2, throat_log[:65], width, label='Throat-Clearing', color='orange', alpha=0.7)
axes[1].set_title('Audio Features Only (65 features, Log-Scaled)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Feature Index', fontsize=11)
axes[1].set_ylabel('Log-Scaled Feature Value', fontsize=11)
axes[1].legend(fontsize=10)
axes[1].grid(alpha=0.3, axis='y')
axes[1].axhline(0, color='black', linestyle='-', linewidth=0.5)
# Add labels for MFCC, Spectral, Time-domain regions
axes[1].text(26, axes[1].get_ylim()[1] * 0.85, 'MFCC (52)', ha='center', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))
axes[1].text(57, axes[1].get_ylim()[1] * 0.85, 'Spectral (10)', ha='center', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.4))
axes[1].text(63, axes[1].get_ylim()[1] * 0.85, 'Time (3)', ha='center', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.4))

# Panel 3: IMU features only (last 40)
x_imu = np.arange(40)
axes[2].bar(x_imu - width/2, cough_log[65:], width, label='Cough', color='steelblue', alpha=0.7)
axes[2].bar(x_imu + width/2, throat_log[65:], width, label='Throat-Clearing', color='orange', alpha=0.7)
axes[2].set_title('IMU Features Only (40 features, Log-Scaled)', fontsize=13, fontweight='bold')
axes[2].set_xlabel('Feature Index', fontsize=11)
axes[2].set_ylabel('Log-Scaled Feature Value', fontsize=11)
axes[2].legend(fontsize=10)
axes[2].grid(alpha=0.3, axis='y')
axes[2].axhline(0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

# Compute overall statistics using raw values
differences_all = np.abs(complete_features_cough - complete_features_throat)
top_15_indices = np.argsort(differences_all)[-15:][::-1]

# Map feature indices to types
feature_type_map = {
    (0, 52): "MFCC",
    52: "Spectral Centroid",
    53: "Spectral Rolloff", 
    54: "Spectral Bandwidth",
    55: "Spectral Flatness",
    56: "Spectral Contrast",
    57: "Total Power",
    58: "Dominant Frequency",
    59: "Spectral Spread",
    60: "Spectral Skewness",
    61: "Spectral Kurtosis",
    62: "Zero-Crossing Rate",
    63: "RMS Energy",
    64: "Crest Factor",
}

print("\n" + "="*90)
print("📊 TOP 15 MOST DISCRIMINATIVE FEATURES (out of 105)")
print("="*90)
print(f"{'Rank':<6} {'Type':<10} {'Feature Name':<30} {'Cough':>15} {'Throat':>15}")
print("-" * 90)
for rank, idx in enumerate(top_15_indices, 1):
    if idx < 65:
        feat_type = "Audio"
        if idx < 52:
            feat_name = f"MFCC #{idx}"
        else:
            feat_name = feature_type_map.get(idx, f"Audio #{idx}")
    else:
        feat_type = "IMU"
        imu_idx = idx - 65
        sig_idx = imu_idx // 5
        feat_idx = imu_idx % 5
        feat_name = f"{signal_names_short[sig_idx]}_{feature_types[feat_idx]}"
    
    print(f"{rank:<6} {feat_type:<10} {feat_name:<30} {complete_features_cough[idx]:>15.2f} "
          f"{complete_features_throat[idx]:>15.2f}")

#### 🎯 ข้อมูลเชิงลึกที่สำคัญจากตัวอย่างนี้

1. **ฟีเจอร์เสียงมีอิทธิพลเหนือกว่าในการแยกแยะ** – 15 อันดับแรกเป็นฟีเจอร์เสียงทั้งหมด โดยไม่มี IMU อยู่ใน 15 อันดับแรกเลย!

2. **Spectral features >> MFCC features**:
 - Spectral Kurtosis (อันดับ 1): วัด 'ความแหลมคม' ของการกระจายความถี่
 - Spectral Skewness (อันดับ 2): วัดความไม่สมมาตรของการกระจายความถี่
 - Spectral Rolloff (อันดับ 3): สูงกว่าสำหรับการไอถึง 6 เท่า (2547 เทียบกับ 414 Hz)
 - Spectral Centroid (อันดับ 4): สูงกว่าสำหรับการไอถึง 3 เท่า (988 เทียบกับ 312 Hz)

3. **เหตุใด Spectral features จึงชนะในตัวอย่างนี้**:
 - นี่คือสภาพแวดล้อมที่เงียบ (การนั่ง, ไม่มีเสียงรบกวนรอบข้าง)
 - การไอมีการระเบิดของ Spectrum ที่คมชัดและเป็น Broadband (Kurtosis/Skewness สูง)
 - เสียงกระแอมมีพลังงานความถี่ต่ำที่กระจุกตัว
 - ความแตกต่างของ Spectrum นั้นชัดเจนและสามารถแยกออกจากกันได้ง่าย

4. **บริบทมีความสำคัญ - นี่คือตัวอย่างเดียว**:
 - ในสภาพแวดล้อมที่มีเสียงรบกวน (เพลง, การจราจร) ฟีเจอร์ IMU จะมีความสำคัญมากขึ้น
 - Multimodal (เสียง + IMU) ให้ความแข็งแกร่งในทุกสภาวะ
 - Notebook นี้แสดงการสกัดฟีเจอร์ โมเดลจริงใช้ฟีเจอร์ทั้งหมด 105 ตัว

5. **ขั้นตอนต่อไป**: ฝึก XGBoost บนชุดข้อมูลทั้งหมด → Model_Training_XGBoost.ipynb
 - คุณจะเห็นว่าความสำคัญของฟีเจอร์เปลี่ยนแปลงไปอย่างไรใน Subject 15 คนและทุกสภาวะ
 - โมเดลเรียนรู้ว่าจะต้องอาศัยเสียง, IMU หรือทั้งสองอย่างเมื่อใด

---

## 📚 สรุปและขั้นตอนถัดไป

### สิ่งที่เราบรรลุผล

ใน Notebook นี้ เราได้แปลงข้อมูล Biosignal ดิบให้เป็นฟีเจอร์ที่พร้อมสำหรับ Machine Learning:

**Raw Data (ข้อมูลดิบ) → Features (ฟีเจอร์):**
- **Input**: Window 0.4 วินาที (ตัวอย่างเสียง 6,400 ตัวอย่าง + ตัวอย่าง IMU 40 ตัวอย่างต่อ Window)
- **Output**: 105 ฟีเจอร์ที่กระชับต่อ Window
- **Compression (การบีบอัด)**: จากค่าประมาณ 6,440 ค่า → 105 ค่า (ลดลง 61 เท่า!)

### การแจกแจงฟีเจอร์

| หมวดหมู่ | จำนวน | วัตถุประสงค์ |
|----------|-------|---------|
| **MFCC** | 52 | จับพื้นผิว/สีของเสียง |
| **Spectral** | 10 | อธิบายการกระจายความถี่ |
| **Time-domain** | 3 | วัดความดังและความคมชัด |
| **IMU** | 40 | จับรูปแบบการเคลื่อนไหวของร่างกาย |
| **TOTAL** | **105** | ลายเซ็น Multimodal ที่สมบูรณ์ |

### ข้อมูลเชิงลึกที่สำคัญจากตัวอย่างของเรา

จากการทำตามขั้นตอนทีละขั้นตอน เราได้เรียนรู้ว่า:

1. **การไอมีลักษณะที่แตกต่างกัน:**
   - คลื่นเสียงที่คมชัดและกระแทกกระทั้น (Crest Factor สูง)
   - เนื้อหาความถี่สูงที่รุนแรง (Spectral Centroid/Rolloff สูง)
   - รูปแบบการเคลื่อนไหวที่แตกต่างกัน (Accelerometer + Gyroscope Spikes)

2. **การตรวจจับแบบ Multimodal นั้นมีประสิทธิภาพ:**
   - เสียงอย่างเดียว: ดี แต่ใช้งานลำบากในสภาพแวดล้อมที่มีเสียงรบกวน
   - IMU อย่างเดียว: จับการเคลื่อนไหวได้ แต่ขาดรายละเอียดทางเสียง
   - รวมกัน: การตรวจจับที่แข็งแกร่งในทุกสภาวะ

3. **ฟีเจอร์ทำให้ Machine Learning เป็นไปได้:**
   - แต่ละ Window กลายเป็นจุดในพื้นที่ 105 มิติ
   - การไอจะรวมกลุ่มกันในพื้นที่หนึ่ง ส่วนเสียงที่ไม่ใช่การไอจะรวมกลุ่มกันในอีกพื้นที่หนึ่ง
   - โมเดล ML เรียนรู้ที่จะค้นหาขอบเขตการตัดสินใจที่เหมาะสมที่สุด
---

**ตอนนี้คุณเข้าใจ Pipeline การสกัดฟีเจอร์ที่สมบูรณ์แล้ว!** 🎉

การเดินทางจาก Biosignals ดิบ → Features → โมเดลที่ฝึกฝนแล้ว เป็น Workflow พื้นฐานในการประมวลผลสัญญาณชีวการแพทย์และ Machine Learning ทักษะที่คุณได้เรียนรู้ที่นี่สามารถนำไปใช้กับโดเมนอื่น ๆ ได้อีกมากมาย: การรู้จำคำพูด, การรู้จำกิจกรรม, การวิเคราะห์ ECG, และอื่น ๆ